# EmotiWave: Multilingual Sentiment & Theme Analysis of Social Media Reactions to Creative Content

🎶 *A real-world NLP pipeline using BERT and SentenceTransformers to analyze emotional engagement with a Facebook music video.*

---

#### 🎵 Original Music Video

This project analyzes viewer reactions to my singing performance, posted on Facebook. You can watch the original video by clicking this vibrant link below:

👉 [**🎥 Watch My Singing Performance on Facebook**](https://www.facebook.com/sriyanka.sarkar.5/videos/722454450156983)

The comments used in this analysis were collected directly from this post, making the emotional insights deeply personal and authentic.


#### 📊 Project Overview

I built a project called **EmotiWave** that analyzes emotional engagement with creative content. I collected viewer comments from my music video posted on Facebook, and used NLP to classify both sentiment and thematic resonance.

- For **sentiment analysis**, I used a multilingual BERT model to rate emotional tone from 1 to 5 stars.
- For **theme classification**, I used semantic similarity with SentenceTransformers to match comments to categories like *Artistic Praise* or *Nostalgia*.

The result is a structured emotional map of audience reactions, which I visualized in a clean table. This project demonstrates my ability to apply AI to real-world, multilingual social data and extract meaningful insights from emotionally rich content.


### Step 1: Install Required Libraries

In [2]:
!pip install transformers sentence-transformers torch


Defaulting to user installation because normal site-packages is not writeable


### Step 2: Prepare Comment Dataset

In [4]:
comments_data = [
    {"username": "Biki Roy", "comment": "Woooow your singing is always blessing"},
    {"username": "Shekhar Majumdar", "comment": "Gorgeous"},
    {"username": "Masum Sabnam", "comment": "Khub sundar. kmn achs Sriyanka?"},
    {"username": "Manosi Mandal", "comment": "Onk din por tomar gan sunlam didi .. hostel room er kotha mone pore gelo .. kotooo gan sunechi tomar theke koto moja korechilm ❤️🤩"},
]


### Step 3: Load NLP Models

In [6]:
!pip install huggingface_hub[hf_xet]
# or
#pip install hf_xet


Defaulting to user installation because normal site-packages is not writeable


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import torch


In [8]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(options=['English', 'Bengali'], description='Language:')
display(dropdown)


Dropdown(description='Language:', options=('English', 'Bengali'), value='English')

In [9]:
import requests
requests.get("https://huggingface.co").status_code


200

In [10]:
# Translation
translation_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
translation_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

try:
    translation_model = AutoModelForSeq2SeqLM.from_pretrained(
        "facebook/nllb-200-distilled-600M", local_files_only=True
    )
    translation_tokenizer = AutoTokenizer.from_pretrained(
        "facebook/nllb-200-distilled-600M", local_files_only=True
    )
except Exception as e:
    print("Translation model failed to load:", e)

try:
    sentiment_model = AutoModelForSequenceClassification.from_pretrained(
        "cardiffnlp/twitter-xlm-roberta-base-sentiment", local_files_only=True
    )
    sentiment_tokenizer = AutoTokenizer.from_pretrained(
        "cardiffnlp/twitter-xlm-roberta-base-sentiment", local_files_only=True
    )
except Exception as e:
    print("Sentiment model failed to load:", e)

try:
    topic_model = SentenceTransformer("all-MiniLM-L6-v2", local_files_only=True)
except Exception as e:
    print("Topic model failed to load:", e)


Sentiment model failed to load: 'NoneType' object has no attribute 'endswith'


In [12]:
# Theme Classification
topic_model = SentenceTransformer("all-MiniLM-L6-v2")


In [13]:
topic_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Step 4: Define Your Emotional Themes

In [15]:
topic_dict = {
    "Emotional Impact": "Touched my heart",
    "Artistic Praise": "Beautiful voice and singing",
    "Lyrics & Composition": "Lyrics are deep and meaningful",
    "Nostalgia": "Reminds me of old memories",
    "Other": "General comment or unclear"
}


### Step 5: Define the Theme Classification Function

In [17]:
from sentence_transformers import util

def categorize_topic(text, topic_dict):
    topic_texts = list(topic_dict.values())
    topic_keys = list(topic_dict.keys())
    embeddings = topic_model.encode(topic_texts, convert_to_tensor=True)
    comment_embed = topic_model.encode(text, convert_to_tensor=True)
    cos_scores = util.cos_sim(comment_embed, embeddings)[0]
    top_idx = torch.argmax(cos_scores).item()
    return topic_keys[top_idx], round(cos_scores[top_idx].item(), 3)


### Step 6: Run Theme Classification on Your Comments

In [19]:
for entry in comments_data:
    user = entry["username"]
    original = entry["comment"]
    theme, score = categorize_topic(original, topic_dict)
    
    print(f"👤 {user}")
    print(f"💬 Comment: {original}")
    print(f"🎭 Theme: {theme} ({score})")
    print("-" * 50)


C:\Users\sriya\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


👤 Biki Roy
💬 Comment: Woooow your singing is always blessing
🎭 Theme: Artistic Praise (0.545)
--------------------------------------------------
👤 Shekhar Majumdar
💬 Comment: Gorgeous
🎭 Theme: Artistic Praise (0.398)
--------------------------------------------------
👤 Masum Sabnam
💬 Comment: Khub sundar. kmn achs Sriyanka?
🎭 Theme: Artistic Praise (0.141)
--------------------------------------------------
👤 Manosi Mandal
💬 Comment: Onk din por tomar gan sunlam didi .. hostel room er kotha mone pore gelo .. kotooo gan sunechi tomar theke koto moja korechilm ❤️🤩
🎭 Theme: Nostalgia (0.09)
--------------------------------------------------


In [20]:
import pandas as pd

# Collect results
results = []
for entry in comments_data:
    user = entry["username"]
    original = entry["comment"]
    theme, score = categorize_topic(original, topic_dict)
    
    results.append({
        "User": user,
        "Comment": original,
        "Theme": theme,
        "Theme Score": score
    })

# Convert to DataFrame and display
df = pd.DataFrame(results)
display(df)


,User,Comment,Theme,Theme Score
0,Biki Roy,Woooow your singing is always blessing,Artistic Praise,0.545
1,Shekhar Majumdar,Gorgeous,Artistic Praise,0.398
2,Masum Sabnam,Khub sundar. kmn achs Sriyanka?,Artistic Praise,0.141
3,Manosi Mandal,Onk din por tomar gan sunlam didi .. hostel ro...,Nostalgia,0.090


### Step 7: Load Sentiment Model

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load sentiment model (no tiktoken dependency)
sentiment_tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Define sentiment analysis function
def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = sentiment_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)[0]
    rating = torch.argmax(scores).item() + 1  # Ratings: 1 to 5
    return f"{rating} stars", round(scores.max().item(), 3)


model.safetensors:  83%|########3 | 556M/669M [00:00<?, ?B/s]

C:\Users\sriya\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sriya\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### Step 9: Combine Sentiment & Theme in Final Table

In [22]:
results = []
for entry in comments_data:
    user = entry["username"]
    original = entry["comment"]
    
    sentiment, sentiment_score = analyze_sentiment(original)
    theme, theme_score = categorize_topic(original, topic_dict)
    
    results.append({
        "User": user,
        "Comment": original,
        "Sentiment": sentiment,
        "Sentiment Score": sentiment_score,
        "Theme": theme,
        "Theme Score": theme_score
    })

df = pd.DataFrame(results)
display(df)


C:\Users\sriya\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,User,Comment,Sentiment,Sentiment Score,Theme,Theme Score
0,Biki Roy,Woooow your singing is always blessing,5 stars,0.708,Artistic Praise,0.545
1,Shekhar Majumdar,Gorgeous,5 stars,0.838,Artistic Praise,0.398
2,Masum Sabnam,Khub sundar. kmn achs Sriyanka?,1 stars,0.353,Artistic Praise,0.141
3,Manosi Mandal,Onk din por tomar gan sunlam didi .. hostel ro...,1 stars,0.449,Nostalgia,0.090


---